In [25]:
import os 
from dotenv import load_dotenv

load_dotenv()

True

In [26]:
os.environ["OPEN_AI_KEY"] = os.getenv("OPEN_AI_KEY")
os.environ["OPENAI_API_VERSION"] = os.getenv("OPENAI_API_VERSION")
os.environ["LANGCHAIN_API_KEY"] = os.getenv("LANGCHAIN_API_KEY")
os.environ["LANGCHAIN_PROJECT"] = os.getenv("LANGCHAIN_PROJECT")
os.environ["LANGCHAIN_TRACING_V2"] = "true"

In [27]:
from langchain_openai import AzureChatOpenAI

llm = AzureChatOpenAI(model="gpt-4o", api_key=os.getenv("OPEN_AI_KEY"), api_version=os.getenv("OPENAI_API_VERSION"))

In [28]:
from langgraph.graph import START, END, StateGraph, MessagesState
from langchain_core.messages import RemoveMessage, AIMessage, HumanMessage

def filter_node(state:MessagesState):
    delete_messages = [RemoveMessage(id=m.id) for m in state["messages"][:-2]]
    return {"messages":delete_messages}

def chat_node(state:MessagesState):
    return ({"messages":llm.invoke(state["messages"])})


graph  = StateGraph(MessagesState)

graph.add_node("filter_node", filter_node)
graph.add_node("chat_node", chat_node)

graph.add_edge(START,"filter_node")
graph.add_edge("filter_node","chat_node")
graph.add_edge("chat_node",END)

graph  = graph.compile()




In [29]:
# Message list with a preamble
messages = [AIMessage("Hi.", name="Bot", id="1")]
messages.append(HumanMessage("Hi.", name="Lance", id="2"))
messages.append(AIMessage("So you said you were researching ocean mammals?", name="Bot", id="3"))
messages.append(HumanMessage("Yes, I know about whales. But what others should I learn about?", name="Lance", id="4"))


In [30]:
output = graph.invoke({'messages': messages})
for m in output['messages']:
    m.pretty_print()

================================== Ai Message ==================================
Name: Bot

So you said you were researching ocean mammals?
================================ Human Message =================================
Name: Lance

Yes, I know about whales. But what others should I learn about?
================================== Ai Message ==================================

There are numerous types of ocean mammals that you might find fascinating! Here are a few notable ones:

1. **Dolphins**: Highly intelligent and social creatures known for their playful behavior and complex vocalizations.

2. **Seals and Sea Lions**: Members of the pinniped family, these mammals are adept swimmers and spend a significant amount of time on land as well.

3. **Manatees and Dugongs**: Often referred to as sea cows, these gentle giants graze on underwater vegetation in coastal waters and rivers.

4. **Otters**: Including the Sea Otter, these mammals are known for their use of tools to open shells and

Trim Messages

In [33]:
from langchain_core.messages import trim_messages

# Node
def chat_model_node(state: MessagesState):
    messages = trim_messages(
            state["messages"],
            max_tokens=100,
            strategy="last",
            token_counter=llm,
            allow_partial=False,
        )
    return {"messages": [llm.invoke(messages)]}

# Build graph
builder = StateGraph(MessagesState)
builder.add_node("chat_model", chat_model_node)
builder.add_edge(START, "chat_model")
builder.add_edge("chat_model", END)
graph = builder.compile()



In [ ]:
messages_out_trim = graph.invoke({'messages': messages})